In [38]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [39]:
pair = "BTC_USD"
granularity = "1m"
df_raw = pd.read_csv(utils.get_his_data_filename(pair, granularity))

In [40]:
non_cols = ['open_time', 'close_time','volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [41]:
SLOSS = 0.4 #stoploss
TPROFIT = 0.8 #take profit
ENTRY_PRC = 0.1 #entry procentage

def direction(row):
    if row.close > row.open:
        return 1
    else:
        return -1
    
def get_signal(row):
    if row.high_prev > row.high and row.low_prev < row.low:
        return row.DIRECTION_prev
    return 0


def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.high_prev
    elif row.SIGNAL == -1:
        return row.low_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0
    
def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.high_prev - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:
        return row.low_prev + (row.RANGE_prev * SLOSS)
    else: 
        return 0
    
def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.high_prev + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:
        return row.low_prev - (row.RANGE_prev * TPROFIT)
    else: 
        return 0
    

In [42]:
df = df_raw[['open_time', 'open', 'high', 'low', 'close']].copy()
df['RANGE'] = df.high - df.low
df['high_prev'] = df.high.shift(1)
df['low_prev'] = df.low.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)


(5541, 12)

In [44]:
df_plot = df.iloc[200:350]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1]

In [45]:
fig = go.Figure()

fig.add_trace(go.Candlestick(   #Adding candlestick chart
    x=df_plot.open_time, open=df_plot.open, high=df_plot.high, low=df_plot.low, close=df_plot.close,
    line=dict(width=1), opacity=1,
    increasing_line_color='#2EC866', 
    decreasing_line_color='#FF3A4C',
    increasing_fillcolor='#24A06B', 
    decreasing_fillcolor='#CC2E3B',
    )) 

fig.add_trace(go.Scatter(
    x=df_buys.open_time,
    y=df_buys.close,
    mode='markers',
    marker=dict(color='#0ec4ce', size=12)
))

fig.add_trace(go.Scatter(
    x=df_sells.open_time,
    y=df_sells.close,
    mode='markers',
    marker=dict(color='#043ef9', size=12)
))

fig.update_layout(width=1100, height=450,
                  margin=dict(l=30, r=30, t=30, b=30),
                  font=dict(color='#e1e1e1'), 
                  paper_bgcolor='#1e1e1e',
                  plot_bgcolor='#1e1e1e')

fig.update_xaxes(
    gridcolor='#1f292f',
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
    #rangebreaks=[dict(bounds=["sat", "mon"])]
)
 
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True
)
fig.show()